En este cuaderno se va a proceder a estudiar la posibilidad de usar en el futuro modelo las columnas del dataset principal que hacen referencia a la media y a la mediana de tiempo jugado por videojuego. 

Ya que en el dataset principal, los registros que cuentan con estos datos son pocos, vamos a proceder a intentar conseguir complementar los registros que no tengan estos datos con la ayuda de un dataset de apoyo.

## 0. Librerías

In [ ]:
# Carga y manipulación de datos
from google.colab import files
import io
 
# DataFrame librery
import pandas as pd

## 1. Carga de datos

Se cargarán dos dataset, **steam_games**, que hace referencia al dataset principal, y **hours_played**, el dataset de apoyo del que intentaremos extraer los máximos datos posibles.

In [ ]:
steam_games = pd.read_csv('steam.csv', sep=',') ## Dataset principal
hours_played = pd.read_csv('hours_played.csv', sep=',',quotechar='"') ## Dataset de apoyo

## 2. Dataset principal

In [ ]:
print("Número de registros: " + str(len(steam_games)))
steam_games.head()

Número de registros: 27075


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


Mostramos cuantas filas cuentan con valor 0 para la media y la mediana.

In [ ]:
len(steam_games[steam_games.average_playtime == 0])

20905

In [ ]:
len(steam_games[steam_games.median_playtime == 0])

20905

Serán de estos 20905 registros, de los cuales intentaremos encontrar su media y mediana en el dataset de apoyo.

## 3. Dataset de apoyo

Este dataset cuenta con un registro de las horas jugadas por diversos usuarios a diferentes juegos, agrupando estas horas podremos calcular media y mediana respecto a estos datos.

In [ ]:
print("Número de registros: " + str(len(hours_played)))
hours_played.head()

Número de registros: 200000


,usuario,nombre,tipo,horas,0
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


### Limpieza del dataset

Depuramos el dataset, eliminando columnas innecesarias y registros que no tienen un valor real.

In [ ]:
## Columnas que no aportan nada completas.
hours_played.drop("0", axis = 1, inplace = True) ## Columna vacía por defecto.
hours_played.drop('usuario', axis = 1, inplace = True) 

## De la columna tipo, solo nos importan las filas con valor 'play'. El resto ('purchase') se refiere a compras del juego.
hours_played = hours_played.loc[hours_played.tipo == 'play'] # Nos quedamos solo con estas filas.
hours_played.drop('tipo', axis = 1, inplace = True) # Y ahora si podemos eliminar esta columna.

hours_played

,nombre,horas
1,The Elder Scrolls V Skyrim,273.0
3,Fallout 4,87.0
5,Spore,14.9
7,Fallout New Vegas,12.1
9,Left 4 Dead 2,8.9
...,...,...
199991,Fallen Earth,2.4
199993,Magic Duels,2.2
199995,Titan Souls,1.5
199997,Grand Theft Auto Vice City,1.5


De los 200000 registros iniciales, tras eliminar los que eran de tipo purchase, nos quedan 70489 registros.

Ahora, como en el dataset principal las variables que queremos rellenar son la media y la mediana, agrupamos los registros del dataset de apoyo por ambas.

### Cálculo de la media

In [ ]:
# Agrupamos por media
groupby_media = hours_played.groupby('nombre').mean()
groupby_media

,horas
nombre,
007 Legends,0.700000
0RBITALIS,0.400000
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),4.000000
10 Second Ninja,2.950000
"10,000,000",3.600000
...,...
rymdkapsel,1.100000
sZone-Online,0.977586
the static speaks my name,0.250000


### Cálculo de la mediana

In [ ]:
# Agrupamos por mediana
groupby_mediana = hours_played.groupby('nombre').median()
groupby_mediana

,horas
nombre,
007 Legends,0.70
0RBITALIS,0.30
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),2.40
10 Second Ninja,2.95
"10,000,000",3.60
...,...
rymdkapsel,1.10
sZone-Online,0.60
the static speaks my name,0.25


### Cálculo de la desviación típica

Además, también vamos a calcular la desviación típica, que nos ofrecerá información sobre la dispersión media de cada juego.

Los grados de libertad se establecen a 0, ya que en el caso de que solo haya un registro por juego, el valor que aplicará será 0, en vez de dejar el valor vacío (NaN).

In [ ]:
# Agrupamos por desviación típica
groupby_std = hours_played.groupby('nombre').std(ddof=0)
groupby_std

,horas
nombre,
007 Legends,0.000000
0RBITALIS,0.141421
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),3.941573
10 Second Ninja,2.450000
"10,000,000",0.000000
...,...
rymdkapsel,0.000000
sZone-Online,1.253690
the static speaks my name,0.122474


## 4. Comparación de ambos datasets

Al no contar en este dataset de apoyo con un id que le relacione de forma directa con el dataset principal, intentaremos identificar los juegos por el nombre.
 
Se va a comprobar dos cosas, la primera, ver cuantos juegos del dataset de apoyo coinciden con los del dataset principal. Y segundo, cuantos de los que coinciden, tienen valor 0 en media y mediana, y por tanto, se pueden rellenar con los datos del dataset de apoyo.

Además de todos los juegos que coincidan, independientemente del valor de media y mediana, se le aportará el valor de su desviación típica, ya que esta columna no está en el dataset principal.

In [ ]:
contador = 0
contador2 = 0
contador3 = 0
for i in groupby_media.index.values:
  for y in steam_games.index:
    if i == steam_games["name"][y]:
      contador = contador + 1 ## Existen en el dataset principal
      if steam_games["average_playtime"][y] == 0:
        contador2 = contador2 + 1
      if steam_games["median_playtime"][y] == 0:
        contador3 = contador3 + 1

print("Juegos que coinciden con los del dataset principal: " + str(contador)) # 1714 son los juegos que coinciden en nombre con los del dataset. 
# Pero, ¿cuáles de esos tienen valor 0 en el dataset principal? 
print("De los juegos que coinciden, tiene valor 0 en la media en el dataset principal: " + str(contador2))
print("De los juegos que coinciden, tiene valor 0 en la mediana en el dataset principal: " + str(contador3))

Juegos que coinciden con los del dataset principal: 1724
De los juegos que coinciden, tiene valor 0 en la media en el dataset principal: 416
De los juegos que coinciden, tiene valor 0 en la mediana en el dataset principal: 416


Se procede a rellenar los juegos que coinciden entre ambos datasets y que en el principal el valor tiene 0. 

Para ello, creamos un dataset nuevo, que contenga solo las filas que serán usadas posteriormente en el dataset principal.

In [ ]:
df_auxiliar = pd.DataFrame(columns=['appid','average_playtime','median_playtime','standard_deviation'])
df_auxiliar['appid'] = steam_games['appid']



### Columna media



In [ ]:
listaMedia = []
contador1 = 0 # Valor no válido en dataset principal y encontrado en el dataset de apoyo
contador2 = 0 # Valor no válido en dataset principal y no encontrado en el dataset de apoyo
contador3 = 0 # Valor válido en dataset principal

for i in steam_games.index:
  if steam_games["average_playtime"][i] == 0: # Si el juego no tiene un valor válido de la media en el dataset principal.
    encontrado = False # Inicializamos en cada iteración a false 
    for y in groupby_media.index.values:
        if y == steam_games["name"][i]: # Si podemos encontrar el juego en el dataset de apoyo, guardamos el dato suyo.
          listaMedia.append(groupby_media.horas[y])
          contador1 = contador1 + 1
          encontrado = True
          continue # Si lo encontramos, pasamos a la siguiente iteración del bucle for.

    if (encontrado == False): # Si no podemos encontrar el juego en el dataset de apoyo, lo dejamos como estaba en el dataset principal.
          listaMedia.append(steam_games["average_playtime"][i])
          contador2 = contador2 + 1
  else: # Si el juego tiene un valor válido para la media en el dataset principal, lo guardamos.
    listaMedia.append(steam_games["average_playtime"][i])
    contador3 = contador3 + 1

print("Juegos con valor no válido en el dataset principal y encontrado en el dataset de apoyo: " + str(contador1))
print("Juegos con valor no válido en el dataset principal y no encontrado en el dataset de apoyo: " + str(contador2))
print("Juegos con valor válido en el dataset principal: " + str(contador3)) 
df_auxiliar['average_playtime'] = listaMedia ## Creación nueva columna

Juegos con valor no válido en el dataset principal y encontrado en el dataset de apoyo: 416
Juegos con valor no válido en el dataset principal y no encontrado en el dataset de apoyo: 20489
Juegos con valor válido en el dataset principal: 6170


### Columna mediana

In [ ]:
listaMediana = []
contador1 = 0 # Valor no válido en dataset principal y encontado en el dataset de apoyo
contador2 = 0 # Valor no válido en dataset principal y no encontado en el dataset de apoyo
contador3 = 0 # Valor válido en dataset principal

for i in steam_games.index:
  if steam_games["median_playtime"][i] == 0: # Si el juego no tiene un valor válido de la media en el dataset principal.
    encontrado = False # Inicializamos en cada iteración a false 
    for y in groupby_mediana.index.values:
        if y == steam_games["name"][i]: # Si podemos encontrar el juego en el dataset de apoyo, guardamos el dato suyo.
          listaMediana.append(groupby_mediana.horas[y])
          contador1 = contador1 + 1
          encontrado = True
          continue # Si lo encontramos, pasamos a la siguiente iteración del bucle for.

    if (encontrado == False): # Si no podemos encontrar el juego en el dataset de apoyo, lo dejamos como estaba en el dataset principal.
          listaMediana.append(steam_games["median_playtime"][i])
          contador2 = contador2 + 1        
  else: # Si el juego tiene un valor válido para la media en el dataset principal, lo guardamos.
    listaMediana.append(steam_games["median_playtime"][i])
    contador3 = contador3 + 1

    
print("Juegos con valor no válido en el dataset principal y encontrado en el dataset de apoyo: " + str(contador1))
print("Juegos con valor no válido en el dataset principal y no encontrado en el dataset de apoyo: " + str(contador2))
print("Juegos con valor válido en el dataset principal: " + str(contador3))
df_auxiliar['median_playtime'] = listaMediana ## Creación nueva columna

Juegos con valor no válido en el dataset principal y encontrado en el dataset de apoyo: 416
Juegos con valor no válido en el dataset principal y no encontrado en el dataset de apoyo: 20489
Juegos con valor válido en el dataset principal: 6170


### Columna desviación tipica
La diferencia a la hora de calcular los valores de esta columna respecto a la media y la mediana, es que es un valor que no está en el dataset principal, y por tanto de todos los juegos que se identifiquen por nombre, se calculará su desviación típica.

In [ ]:
listaDesvTipica = []
contador = 0

for i in steam_games.index:
  encontrado = False
  for y in groupby_std.index.values:
    if y == steam_games["name"][i]:
      listaDesvTipica.append(groupby_std.horas[y])
      contador = contador + 1
      encontrado = True
      continue
      
  if encontrado == False:
      listaDesvTipica.append(0)

print("Juegos con desviación típica calculada: " + str(contador))
df_auxiliar['standard_deviation'] = listaDesvTipica ## Creación nueva columna

Juegos con desviación típica calculada: 1724


Ahora, ya tenemos los valores de media, mediana y desviación típica de todos los videojuegos del dataset principal que se han podido conseguir gracias al dataset de apoyo.

In [ ]:
df_auxiliar = df_auxiliar.round({"standard_deviation" : 2})  # Redondeo a 2 decimales
df_auxiliar

,appid,average_playtime,median_playtime,standard_deviation
0,10,17612.0,317.0,617.95
1,20,277.0,62.0,1.13
2,30,187.0,34.0,45.92
3,40,258.0,184.0,1.63
4,50,624.0,415.0,0.00
...,...,...,...,...
27070,1065230,0.0,0.0,0.00
27071,1065570,0.0,0.0,0.00
27072,1065650,0.0,0.0,0.00
27073,1066700,0.0,0.0,0.00


### Comprobación

Ahora, brevemente se comprueba como un videojuego, que no tiene datos en el dataset principal sobre la media y la mediana, ahora si cuenta con estos datos en nuestro dataset auxiliar.

In [ ]:
steam_games.iloc[26]

appid                                     1002
name                          Rag Doll Kung Fu
release_date                        2005-10-12
english                                      1
developer                          Mark Healey
publisher                          Mark Healey
platforms                              windows
required_age                                 0
categories          Single-player;Multi-player
genres                                   Indie
steamspy_tags       Indie;Fighting;Multiplayer
achievements                                 0
positive_ratings                            40
negative_ratings                            17
average_playtime                             0
median_playtime                              0
owners                             20000-50000
price                                     5.99
Name: 26, dtype: object

In [ ]:
df_auxiliar.iloc[26]

appid                 1002.000
average_playtime         0.475
median_playtime          0.350
standard_deviation       0.390
Name: 26, dtype: float64

Comprobamos que ningún valor del dataset creado este vacío

In [ ]:
pd.isnull(df_auxiliar).any()

appid                 False
average_playtime      False
median_playtime       False
standard_deviation    False
dtype: bool

## 5. Exportando los datos 

Mediante la siguiente función, podremos guardar y exportar los datos calculados en formato CSV, de tal forma, que se puedan cargar en otros cuadernos para seguir con el desarrollo de este estudio.

In [ ]:
df_auxiliar.to_csv('auxiliar3.csv')